In [13]:
#!/usr/local/bin/python3.10
import tensorflow as tf
import numpy as np
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.9.1


In [14]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [15]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D((2,2)),

  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D((2,2)),

  tf.keras.layers.Dropout(0.5), 
  tf.keras.layers.Flatten(input_shape=(4, 4, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [16]:
predictions = model(x_train[:1]).numpy()
predictions

array([[0.10065082, 0.09853759, 0.0982696 , 0.10095433, 0.09983879,
        0.10025577, 0.0979636 , 0.09963704, 0.1022104 , 0.10168211]],
      dtype=float32)

In [17]:
tf.nn.softmax(predictions).numpy()

array([[0.10006502, 0.09985378, 0.09982701, 0.10009539, 0.09998379,
        0.10002549, 0.09979648, 0.09996362, 0.10022119, 0.10016827]],
      dtype=float32)

In [18]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [19]:
loss_fn(y_train[:1], predictions).numpy()

2.30233

In [20]:
optimizerFunction = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.995,
    epsilon=5e-06,
    amsgrad=True,
    name='Adam',
)

In [21]:
model.compile(optimizer=optimizerFunction,
              loss=loss_fn,
              metrics=['accuracy'])

In [22]:
callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=2)
model.fit(x_train, y_train, epochs=15, batch_size=32, callbacks=[callback])

Epoch 1/15


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1875/1875 [==============================] - 106s 56ms/step - loss: 0.2437 - accuracy: 0.9289
Epoch 2/15
1875/1875 [==============================] - 98s 52ms/step - loss: 0.0855 - accuracy: 0.9768
Epoch 3/15
1875/1875 [==============================] - 98s 52ms/step - loss: 0.0627 - accuracy: 0.9833
Epoch 4/15
1875/1875 [==============================] - 89s 47ms/step - loss: 0.0514 - accuracy: 0.9860
Epoch 5/15
1875/1875 [==============================] - 89s 48ms/step - loss: 0.0456 - accuracy: 0.9883
Epoch 6/15
1875/1875 [==============================] - 79s 42ms/step - loss: 0.0375 - accuracy: 0.9895
Epoch 7/15
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0333 - accuracy: 0.9908
Epoch 8/15
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0336 - accuracy: 0.9908
Epoch 9/15
1875/1875 [==============================] - 80s 43ms/step - loss: 0.0277 - accuracy: 0.9918
Epoch 10/15
1875/1875 [==============================] - 82s 44ms/step - l

In [23]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 4s - loss: 0.0218 - accuracy: 0.9934 - 4s/epoch - 13ms/step


[0.021769000217318535, 0.993399977684021]

In [24]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [25]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.08533674, 0.08533674, 0.08533674, 0.08533674, 0.08533674,
        0.08533674, 0.08533674, 0.23196931, 0.08533674, 0.08533674],
       [0.08533674, 0.08533674, 0.23196931, 0.08533674, 0.08533674,
        0.08533674, 0.08533674, 0.08533674, 0.08533674, 0.08533674],
       [0.08533675, 0.23196931, 0.08533675, 0.08533675, 0.08533675,
        0.08533675, 0.08533675, 0.08533675, 0.08533677, 0.08533675],
       [0.23196931, 0.08533674, 0.08533674, 0.08533674, 0.08533674,
        0.08533674, 0.08533674, 0.08533674, 0.08533674, 0.08533674],
       [0.08533689, 0.08533689, 0.08533689, 0.08533689, 0.231967  ,
        0.08533689, 0.08533689, 0.08533689, 0.08533689, 0.08533788]],
      dtype=float32)>

In [26]:
model.save('mnist_predictor.h5')

Reload the model

In [30]:
new_model = tf.keras.models.load_model('mnist_predictor.h5')

new_model.summary()

input_image = np.ones((28, 28, 1))  # Replace this with your actual image data

# Preprocess the image data (optional, depending on how you trained the model)
# input_image = preprocess(input_image)

# Expand the dimensions to match the expected shape (batch_size, 28, 28, 1)
input_image = np.expand_dims(input_image, axis=0)

# Make predictions
predictions = new_model.predict(input_image)
print(predictions)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        4640      
                                                                 
 batch_normalization (BatchN  (None, 24, 24, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         7